In [16]:
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import eigsh

import cvxpy as cvx
import mosek
import time

In [40]:
m = 5
# np.random.seed(1)
A1, b1, c1 = np.random.randn(m,m)*10,np.random.randn(m)*10,5
A1 = A1.T+A1
A2, c2 = np.random.randn(m,m)*10, 10
AA = A2
A2 = A2.T.dot(A2)
# A2 = A2 - eigsh(A2,1,which='SA')[0][0]*np.eye(m)
# A2 = A2.T*A2

In [41]:
def binary_dual_solve(A1,b1,c1,A2,c2):
    # 二分法求对偶问题
    A1_temp, b1_temp = np.insert(A1,m,values=b1,axis=0), np.insert(b1,m,values=c1)
    A = np.insert(A1_temp,m,values=b1_temp,axis=1)

    mu = eigsh(A,1,which='SA')[0][0]
    lambda_min, lambda_max = 0, (c1-mu)/c2
    epsilon = 1e-6

    f, g, u = [], [], []
    k, u0, d_lambda = 0, 6, 6

    while abs(d_lambda)>epsilon and abs(u0)>epsilon:
        k = k+1
        lambda_ = (lambda_min+lambda_max)/2

        A1_temp, b1_temp = np.insert(A1+lambda_*A2,m,values=b1,axis=0), np.insert(b1,m,values=c1-lambda_*c2)
        G = np.insert(A1_temp,m,values=b1_temp,axis=1)

    #     最小特征值和特征向量
        min_e, min_ev = eigsh(G,1,which='SA')
        y = min_ev/min_ev[-1]
        x = y[:-1]
        f1, f2 = y.T.dot(A).dot(y), 1+np.linalg.norm(x)**2
        f0 = f1/f2
        f.append(f0)
        u0 = (x.T.dot(A2).dot(x) - c2)/(f2)
        u.append(u0)
        if u0>0:
            lambda_min = lambda_
        else:
            lambda_max = lambda_
        g0, d_lambda = min_e,lambda_max-lambda_min
        g.append(g0)
#     print((f0[0][0],g0[0]))
#     print(lambda_)
    return (g0[0],lambda_)
    # print(x)

(-17.035665113558704, -17.035667734041873)
1.2051957412645415
[[-0.47582198]
 [-0.05396027]
 [-0.17714114]
 [ 0.14266396]
 [ 0.15873277]]


6

In [63]:
# cvx求对偶问题
# Construct the problem.
t = cvx.Variable()
lambda_ = cvx.Variable()
objective = cvx.Maximize(t)
zero = np.zeros(m)
A = np.insert(np.insert(A1,m,values=b1,axis=0),m,values=np.insert(b1,m,values=c1),axis=1)
B = np.insert(np.insert(A2,m,values=zero,axis=0),m,values=np.insert(zero,m,values=-c2),axis=1)
constraints = [A+lambda_*B-t*np.eye(m+1)>>0, lambda_ >= 0]
prob = cvx.Problem(objective, constraints)

# The optimal objective is returned by prob.solve().
start = time.clock()
result = prob.solve(solver=cvx.CVXOPT)
elapsed = time.clock() - start
# The optimal value for x is stored in x.value.
print(t.value)
# print(prob.value)
print(lambda_.value)
# print(prob.solver_stats.solve_time)
# print(elapsed)
value, lambda_ = t.value,lambda_.value

-17.035667736235773
1.2052054864680095


In [44]:
# 使用cvx求解投影梯度
def subproblem(dh,y,rho,A2,c2):
    x = cvx.Variable(m)
    objctive = cvx.Minimize(rho*cvx.square(cvx.norm(x-y)) + dh*x)
    constrain = [cvx.quad_form(x,A2) <= c2]
    prob = cvx.Problem(objctive,constrain)
    prob.solve(solver=cvx.SCS)
    return x.value
def binary_subproblem(H,b):
    x = cvx.Variable(m)
    objctive = cvx.Minimize(cvx.quad_form(x,H) + b*x)
    prob = cvx.Problem(objctive)
    prob.solve(solver=cvx.SCS)
    return x.value
# 二分法求子问题
def binary_search(dh,y,rho,A2,c2):
    t0 = dh.T.dot(y) - 1/(4*rho)*np.linalg.norm(dh)**2
    eta_l, eta_r = 0, (rho*np.linalg.norm(y)**2 - t0)/c2
    d_eta = eta_r - eta_l
    cx = 6
    I_m = 2*rho*np.eye(m)
    b = dh - 2*rho*y
    while d_eta > epislon and abs(cx) > epislon:
        eta = (eta_l + eta_r)/2
        H = I_m + eta*A2
        x = nersterov_unconstrained(H,b)
#         x = binary_subproblem(H,b)
        cx = x.T.dot(A2).dot(x) - c2
        
        if cx > 0:
            eta_l = eta
        elif cx < 0:
            eta_r = eta
        d_eta = eta_r - eta_l
        print((d_eta,cx))
    return 2*x
def nersterov_unconstrained(H,b):
    rho = eigsh(H,1,which='LA')[0][0]
    y1,x1,theta1 = np.zeros(m),np.zeros(m),1
    df = 6
    while abs(df) > epislon:
        grad_f = 2*H.dot(y1) + b
        x2 = y1 - 1/rho * grad_f
        theta2 = 1/2*(1+np.sqrt(1+4*theta1**2))
        y2 = x2 + (1-theta1)/theta2*(x2-x1)
        df = x2.T.dot(H).dot(x2) + b.T.dot(x2) - (x1.T.dot(H).dot(x1) + b.T.dot(x1))
        x1, y1, theta1 = x2, y2, theta2
#         print(df)
    return x2

In [1]:
# 原论文版本
# 带约束的nersterov加速梯度法求解原问题
H = A1 + lambda_*A2 - value*np.eye(m)
mu, L = eigsh(H,1,which='SA')[0][0], eigsh(H,1,which='LA')[0][0]
q = mu/L
x1 = np.random.randn(m)*10
y = x1
alpha0,alpha1 = 0.9,0.9
dx = 6
epislon = 1e-5
k = 0

while np.linalg.norm(dx) > epislon:
    x0 = x1
    dh = 2*(H.dot(y) + b1)
#     x1 = subproblem(dh,y,L,A2,c2)
    x1 = binary_search(dh,y,L,A2,c2)
    print((np.linalg.norm(dx),x1))
    alpha0 = alpha1
    alpha1 = 1/2*(np.sqrt((alpha0**2 - q)**2 + 4*alpha0**2) - (alpha0**2 - q))
    beta = (alpha0*(1-alpha0))/(alpha0**2 + alpha1)
    y = x1 + beta*(x1 - x0)
    dx = x1 - x0
f1, f2 = x1.T.dot(A1.dot(x1))+2*b1.T.dot(x1)+c1, 1+np.linalg.norm(x1)**2
f0 = f1/f2
print(x1)
print(f0)

In [66]:
?prob.solve()

In [65]:
['']*3

['', '', '']

In [64]:
# 对比
x = cvx.Variable(m)
objctive = cvx.Minimize(cvx.quad_form(x,H) + 2*b1*x)
constrain = [cvx.quad_form(x,A2)-c2 <=0]
prob = cvx.Problem(objctive,constrain)
prob.solve(solver=cvx.SCS)
print(x.value)

[-0.47571798 -0.0539156  -0.17706759  0.14260468  0.15871833]


In [34]:
np.linalg.inv(H).dot(b1)

array([-0.42995839,  0.10048954, -0.00882551, -0.31853316,  0.04634013])

In [12]:
# cvx测试qcqp
H = A1 - value*np.eye(m)
c = c1 - value
from qcqp import *
x = cvx.Variable(m)
objctive = cvx.Minimize(cvx.quad_form(x,H) + 2*b1*x+c)
C = A2 + 3*np.eye(m)
constrain = [cvx.quad_form(x,C)-c2 >= 0]
prob = cvx.Problem(objctive,constrain)
# Create a QCQP handler.
qcqp = QCQP(prob)

# Solve the SDP relaxation and get a starting point to a local method
qcqp.suggest(SDR)
print("SDR lower bound: %.3f" % qcqp.sdr_bound)

# Attempt to improve the starting point given by the suggest method
f_cd, v_cd = qcqp.improve(COORD_DESCENT)
print("Coordinate descent: objective %.3f, violation %.3f" % (f_cd, v_cd))
print(x.value)

In [12]:
# scipy测试qcqp
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint

H = A1 - value*np.eye(m)
c = c1 - value
obj_fun = lambda x : x.T.dot(H).dot(x) + 2*b1.T.dot(x) + c
obj_grad = lambda x : H.dot(x) + 2*b1
obj_hess = lambda x : H

cons_fun = lambda x : x.T.dot(A2).dot(x) - c2
cons_grad = lambda x : A2.dot(x)
cons_hess = lambda x : A2
nonlinear_constraint = NonlinearConstraint(cons_fun,-np.inf,0, jac=cons_grad, hess=cons_hess)
x0 = np.random.randn(m)*10
res = minimize(obj_fun,x0,method='trust-constr',jac=obj_grad,hess=obj_hess,constraints=[ nonlinear_constraint])

TypeError: __init__() got an unexpected keyword argument 'hessp'

In [13]:
import picos as pic
import cvxopt as cvx

H = A1 - value*np.eye(m) + lambda_*A2
c = c1 - value

H = pic.new_param('H',H)
c = pic.new_param('c',c)
b1 = pic.new_param('b1',b1)

prob = pic.Problem()
x = prob.add_variable('x',m)
prob.set_objective('min',x.T*H* x + 2*b1.T* x +c)
prob.solve(verbose=0,solver='cplex')

CPXPARAM_Simplex_Display                         0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             0
CPXPARAM_Barrier_Display                         0
CPXPARAM_Barrier_QCPConvergeTol                  1e-08


{'cplex_solution': <cplex._internal._subinterfaces.SolutionInterface at 0x7f15d81ee8d0>,
 'obj': -5.0807308854858055,
 'status': 'optimal',
 'time': 0.007035017013549805}

In [11]:
pic.tools.available_solvers()

['cvxopt', 'mosek7', 'cplex']

## 系统版本
Linux version 3.10.0-514.16.1.el7.x86_64 (builder@kbuilder.dev.centos.org) (gcc version 4.8.5 20150623 (Red Hat 4.8.5-11) (GCC) ) #1 SMP Wed Apr 12 15:04:24 UTC 2017
## CPU版本

Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz

## 内存
128G

In [68]:
131774344/(2**20)

125.66980743408203